<a href="https://colab.research.google.com/github/AnjanaAbY/Font-Recognition-Model/blob/main/Font_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.applications import VGG16 #pre-trained CNN model that is commonly used for image classification tasks
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load and preprocess the dataset
train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/font_patch',
    target_size=(105, 105),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/font_patch',
    target_size=(105, 105),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    subset='validation')

# Load the pre-trained VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(105, 105, 3))

# Freeze the convolutional base
base_model.trainable = False

# Create a new model on top
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))  # Assuming 6 font classes

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=2,  # Adjusted for the small dataset
    epochs=10
)

# Save the model
model.save('font_recognition_model.h5')

Found 39 images belonging to 6 classes.
Found 7 images belonging to 6 classes.
58889256/58889256 [==============================] - 0s 0us/step


Epoch 1/10
2/2 [==============================] - 5s 721ms/step - loss: 2.2769 - accuracy: 0.1795
Epoch 2/10
2/2 [==============================] - 6s 1s/step - loss: 1.8803 - accuracy: 0.3077
Epoch 3/10
2/2 [==============================] - 4s 725ms/step - loss: 1.7865 - accuracy: 0.3333
Epoch 4/10
2/2 [==============================] - 6s 5s/step - loss: 1.2893 - accuracy: 0.4359
Epoch 5/10
2/2 [==============================] - 4s 3s/step - loss: 1.2737 - accuracy: 0.4872
Epoch 6/10
2/2 [==============================] - 4s 3s/step - loss: 1.1543 - accuracy: 0.5385
Epoch 7/10
2/2 [==============================] - 6s 5s/step - loss: 1.0660 - accuracy: 0.5897
Epoch 8/10
2/2 [==============================] - 4s 717ms/step - loss: 0.7110 - accuracy: 0.7436
Epoch 9/10
2/2 [==============================] - 5s 729ms/step - loss: 0.9632 - accuracy: 0.6410
Epoch 10/10
2/2 [==============================] - 4s 711ms/step - loss: 0.6805 - accuracy: 0.7436


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
# Install Tesseract OCR
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,834 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [4]:
# Import necessary libraries for testing
import keras
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import pytesseract
from PIL import Image

# Load the trained model
model = load_model('font_recognition_model.h5')

# Load and preprocess the new image
img_path = '/content/raleway-regular-preview.png'
img = image.load_img(img_path, target_size=(105, 105))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize the image
img_array = np.vstack([img_array])

# Make predictions for font detection
predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions, axis=1)[0]  # Get the predicted class index

# Assuming font_names is a list where index corresponds to font class
font_names = ['Lato', 'Raleway', 'Roboto', 'Sansation', 'Times New Roman', 'Walkway']

# Get the predicted font name
predicted_font_name = font_names[predicted_class_index]

# Print the predicted font name
print("Predicted Font Name:", predicted_font_name)

# Path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# Load the image from your Colab environment
img = Image.open(img_path)

# Use pytesseract to extract text
text = pytesseract.image_to_string(img)

# Print the extracted text
print("Extracted Text:", text)


1/1 [==============================] - 0s 253ms/step
Predicted Font Name: Raleway
Extracted Text: The quick brown fox jumps over the le

